In [4]:
import ipynb.fs.full.get_artists_info_api as getter

In [5]:
tracks_id = []
for i in range(len(getter.tracks)):
    tracks_id.append(getter.tracks[i]['id'])

tracks_id = list(dict.fromkeys(tracks_id))

In [6]:
track_features_list = []

for i in range(len(tracks_id)):
    track_features_list.append(getter.spotify.audio_features(tracks_id[i]))

In [7]:
#there are some tracks without their audio features so we need to remove them from the list
track_features_list = list(filter(lambda x: x != [None], track_features_list))

In [8]:
for i in range(len(track_features_list)):
    track_features_list[i] = dict(list(filter(lambda x: x[0] == 'id' or type(x[1]) != str, list(track_features_list[i][0].items()))))

In [9]:
artists = list(filter(lambda x: x['genres'] != '' ,getter.artists))

In [10]:
genres_list = []

for i in range(len(artists)):
    keys = ['id', 'genre']
    genre = dict.fromkeys(keys)
    for j in range(len(getter.tracks)):
        if artists[i]['id'] == getter.tracks[j]['artists'][0]['id']:
            genre['id'] = getter.tracks[j]['id']
            genre['genre'] = artists[i]['genres'].split(', ')
            genre['genre'] = genre['genre'][0]
            genres_list.append(genre)

In [11]:
for i in range(len(track_features_list)):
    for j in range(len(genres_list)):
        if track_features_list[i]['id'] == genres_list[j]['id']:
            track_features_list[i]['genre'] = genres_list[j]['genre']

In [12]:
track_features_list = list(filter(lambda x: len(x) == 15, track_features_list))

---------------------------------------------------------------------------------------------------------------

In [68]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [14]:
df = pd.DataFrame(track_features_list)

In [15]:
df.insert(0, 'id', df.pop('id'))

In [16]:
df.head(10)

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre
0,3HWcK5EIfIdOQD1Rsj4yek,0.747,0.422,7,-12.890,0,0.4970,0.005220,0.000831,0.119,0.125,132.128,176364,4,plugg
1,3gZyjgWdEibTsQFEmGiSSk,0.405,0.756,5,-12.469,0,0.0424,0.000188,0.745000,0.130,0.279,160.012,205500,4,speedrun
2,2lFeir1XMPIRPcvNGQwCSr,0.845,0.483,2,-7.859,1,0.2770,0.010900,0.000000,0.136,0.643,129.049,98605,4,pop rap
3,4LRPiXqCikLlN15c3yImP7,0.520,0.731,6,-5.338,0,0.0557,0.342000,0.001010,0.311,0.662,173.930,167303,4,pop
4,4Qum1kECNQrpREev5fvAQ4,0.492,0.708,10,-4.639,1,0.0334,0.097500,0.000000,0.120,0.697,161.989,203037,4,contemporary country
5,2F1Z3G7IZKissAgLtZSUup,0.629,0.726,6,-10.098,0,0.3310,0.186000,0.000000,0.244,0.839,81.148,114000,4,christian trap
6,4C6Uex2ILwJi9sZXRdmqXp,0.950,0.891,2,-2.653,1,0.2410,0.064500,0.000018,0.309,0.912,133.010,170977,4,dance pop
7,4k6Uh1HXdhtusDW5y8Gbvy,0.686,0.494,1,-7.093,1,0.0355,0.613000,0.000058,0.402,0.700,168.946,232067,4,afrofuturism
8,4h4QlmocP3IuwYEj2j14p8,0.768,0.714,10,-5.110,1,0.0401,0.352000,0.000000,0.150,0.842,131.443,163026,4,pop
9,4u2gL1g6AgF2vlAURETOXi,0.429,0.834,0,-6.610,0,0.2610,0.010600,0.000000,0.163,0.664,179.932,110517,4,glitchcore


In [58]:
data = df.drop(['id'], axis=1)

In [59]:
label = data.iloc[:, -1]
label.head(10)

0                   plugg
1                speedrun
2                 pop rap
3                     pop
4    contemporary country
5          christian trap
6               dance pop
7            afrofuturism
8                     pop
9              glitchcore
Name: genre, dtype: object

In [60]:
encoder = LabelEncoder()
y = encoder.fit_transform(label)
y

array([111, 124, 115, 113,  52,  44,  55,   5, 113,  78,  14,  42,  14,
        37,  86, 113,  32,  89,  17,  55,  29,  33, 109,  80,  55,  55,
        36,  55, 113,  70,  82,  59,  55, 105,  35,  94, 110,  51, 113,
        14,  13,  62,  75,  96, 119,  86,  52,   7,  77,  55,  90,  37,
        57, 113,  14,  13,  55,  55,  15,  55,  55, 128,  22,  55,  36,
        16,  34, 112,  29, 111,  11, 122, 120,  34, 103, 125,  28,  23,
        47,  70,  58, 102,  55, 108, 121,  15,  55,  94,  55,  55,  94,
        55,   9,   9, 126,  98,  55, 113,  89,  55,   3,  67,  69, 114,
        55,   6,  74, 116,  20,  76,  12,  10,  89,  55,  98,  19,  45,
        19,  55,  55,  64,   7, 101,  66,  55,  78,  95,  14,  11,  48,
       113,  80,  99,   7,   1,  13,  37,  46,  39,   1,  55,  10,  38,
        55, 104,  37,  25, 100,   1,  79, 118,  30,  73,  71,  55,  35,
        55,  74,  55,  37, 107,  25,   6,  88,  55,  55, 106,  37,  22,
         6,  11,  19,  11,  55,  86,  72,  52, 104, 117,  11,  1

In [61]:
scaler = StandardScaler()
x = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
x.shape

(255, 13)

In [62]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
x_train.shape

(178, 13)

-------

In [63]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import BaggingClassifier

In [71]:
model = BaggingClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

/home/sukivn/anaconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
/home/sukivn/anaconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
/home/sukivn/anaconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [72]:
print(n_scores)
print(n_scores.mean())

[0.11111111 0.05555556 0.11111111 0.16666667 0.05555556 0.27777778
 0.05555556 0.05555556 0.05882353 0.05882353 0.05555556 0.11111111
 0.16666667 0.11111111 0.11111111 0.05555556 0.11111111 0.05555556
 0.17647059 0.11764706 0.16666667 0.11111111 0.05555556 0.22222222
 0.16666667 0.05555556 0.11111111 0.         0.23529412 0.17647059]
0.11263616557734203


In [73]:
model.fit(x_train, y_train)
pred = model.predict(x_test)
pred

array([ 46,  55,   2,   9,  55,  10,  55,  78,   9,  86,  52,  10,  55,
        34,  55,  13,  55,  55,   1,  55,  14, 113,  55,  86,  55,   2,
        55, 121,   1,  37,   8,  55,  55,  10,  55, 113,  31,  66,  71,
        10,  55,  55,   1,  34,  14,  55,  34,  42,  16,  55,  55,  55,
        10,  55,  55,  14, 126,  94,  55,  55,  34,  55,  37,  14,  14,
         6,  55,  42,  55,  55,  55,  55,  55,   0, 113,  55,  10])

In [74]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
accuracy_score(y_test, pred)

0.07792207792207792

In [75]:
f1_score(y_test, pred, average='weighted')

0.04503288918873334